In [3]:
__author__ = 'Tania Reyes - reyes.mtz.tania@gmail.com'

import math
import numpy as np
import cv2
import sys

def cloth_color(image_path, expected_size=40, in_clusters=7, out_clusters=3, final_size=300):
    image_path = "66girls2.jpg"
    initial_image = cv2.imread(image_path)

    if initial_image is not None:
        height, width = initial_image.shape[:2]

        factor = math.sqrt(width * height / (expected_size * expected_size))

        #image downsample
        image = cv2.resize(initial_image,
                           (int(width / factor), int(height / factor)),
                           interpolation=cv2.INTER_LINEAR)

        LAB_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
        frame_width = int(expected_size / 10 + 2)
        in_samples = []

        border_samples = []

        limit_Y = LAB_image.shape[0] - frame_width
        limit_X = LAB_image.shape[1] - frame_width

        for y in range(LAB_image.shape[0] - 1):
            for x in range(LAB_image.shape[1] - 1):
                pt = LAB_image[y][x]
                if x < frame_width or y < frame_width or y >= limit_Y or x >= limit_X:
                    border_samples.append(pt)
                else:
                    in_samples.append(pt)

        in_samples = np.array(in_samples, dtype=float)
        border_samples = np.array(border_samples, dtype=float)

        em_in = cv2.ml.EM_create()
        em_in.setClustersNumber(in_clusters)
        in_etval, in_likelihoods, in_labels, in_probs = em_in.trainEM(in_samples)

        in_means = em_in.getMeans()
        in_covs = em_in.getCovs()

        em_border = cv2.ml.EM_create()
        em_border.setClustersNumber(out_clusters)
        border_etval, border_likelihoods, border_labels, border_probs = em_border.trainEM(border_samples)

        border_means = em_border.getMeans()

        unique_border, counts_border = np.unique(border_labels, return_counts=True)
        count_border_labels = dict(zip(unique_border, counts_border))

        unique, counts = np.unique(in_labels, return_counts=True)

        count_in_labels = dict(zip(unique, counts))
        in_len = len(in_covs)

        valid = [True] * in_len

        # colors vs background
        for i in range(in_len):
            if not valid[i]:
                continue

            prop_in = float(count_in_labels[i]) / len(in_labels)

            #if the proportion is too small can be only buttons or labels
            if prop_in < 0.05:
                valid[i] = False
                continue

            # remove similar colors
            for key in count_border_labels:
                prop_border = float(count_border_labels[key]) / len(border_labels)

                #if the color appears more in the center, it belongs to the cloth
                #else is background
                if prop_in >= prop_border:
                    continue

                cie_dst = CIE2000_distance(in_means[i], border_means[key])

                if cie_dst < 5:
                    valid[i] = False

        # colors vs colors
        for i in range(in_len):
            if not valid[i]:
                continue

            for j in range(i + 1, in_len):
                if not valid[j]:
                    continue

                #removes shadows and similar colors
                cie_dst = CIE2000_distance(in_means[i], in_means[j])

                is_shadow = LAB_shadow(in_means[i], in_means[j])

                if is_shadow or cie_dst < 20:
                    if count_in_labels[j] > count_in_labels[i]:
                        valid[i] = False

                        count_in_labels[j] += count_in_labels[i]
                        break
                    else:
                        valid[j] = False
                        count_in_labels[i] += count_in_labels[j]

        num_valid = sum(True == x for x in valid)

        colors = []
        proportions = []
        total_color = 1

        #if the cloth is of the same color that the background, takes the more common color
        if num_valid == 0:
            pos = max(count_in_labels, key=count_in_labels.get)
            colors = [in_means[pos]]
            proportions = [count_in_labels[pos]]
            total_color = count_in_labels[pos]

        for i in range(in_len):

            if not valid[i]:
                continue

            color = in_means[i]
            quantity = count_in_labels[i]
            colors.append(color)
            proportions.append(quantity)
            total_color += quantity

        factor = max(1,math.sqrt(width * height / (final_size * final_size)))

        final_image = cv2.resize(initial_image,
                                 (int(width / factor),
                                  int(height / factor)),
                                 interpolation=cv2.INTER_LINEAR)
        final_height, final_width = final_image.shape[:2]

        colors_width = int(final_width / 2.0)

        image_with_border = cv2.copyMakeBorder(final_image,
                                               top=0,
                                               bottom=0,
                                               left=0,
                                               right=colors_width,
                                               borderType=cv2.BORDER_CONSTANT,
                                               value=[0.0, 0.0, 0.0])

        y_color = 0

        for i, color_LAB in enumerate(colors):
            color_LAB = np.array([[[color_LAB[0], color_LAB[1], color_LAB[2]]]])
            color_LAB = color_LAB.astype(np.uint8)
            color = cv2.cvtColor(color_LAB, cv2.COLOR_Lab2BGR)[0][0]
            color = color.tolist()
            height_color = int(math.ceil(proportions[i]*final_height/float(total_color)))

            cv2.rectangle(image_with_border,
                          (final_width, y_color),
                          (final_width + colors_width, y_color + height_color),
                          color,
                          -1)
            y_color += height_color
        cv2.imshow("colors", image_with_border)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Image not found")

if __name__ == "__main__":
    cloth_color(sys.argv[1:][0])

NameError: name 'CIE2000_distance' is not defined